In [1]:
import pandas as pd

places_data = pd.read_csv("final-dataset/main_dataset.csv") 
reviews_data = pd.read_csv("final-dataset/review_dataset.csv")  

user_place_reviews = pd.merge(reviews_data,places_data, on='id')

# drop review_number, lat long, address, url,status, phone, types_y, price-level,review 1-5
user_place_reviews = user_place_reviews.drop(columns=['review_number', 'latitude', 'longitude', 'address', 'url', 'status', 'phone', 'types_y', 'price-level', 'review 1', 'review 2', 'review 3', 'review 4', 'review 5'])
user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0


In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

user_place_reviews['types_x'] = user_place_reviews['types_x'].str.split(', ')

mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(user_place_reviews['types_x'])

one_hot_df = pd.DataFrame(one_hot, columns=mlb.classes_)

user_place_reviews = pd.concat([user_place_reviews, one_hot_df], axis=1)

user_place_reviews = user_place_reviews.drop_duplicates(subset=['user_id', 'id'])

# user_place_reviews = user_place_reviews.drop(columns=['types_x'])

user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count,airport,...,tourist_attraction,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo
0,ChIJYcGr7GSb0S0RckePBrCWikw,"[hotel, lodging]","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0,0,...,0,0,0,0,0,0,0,0,0,0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"[indonesian_restaurant, restaurant, food]","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0,0,...,0,0,0,0,0,0,0,0,0,0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"[tourist_attraction, restaurant, food]","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0,0,...,1,0,0,0,0,0,0,0,0,0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"[coffee_shop, cafe, food, store]","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0,0,...,0,0,0,0,0,0,0,0,0,0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"[tourist_attraction, place_of_worship]","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
user_place_reviews.to_csv('final-dataset/user_reviews.csv', index=False)

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# List of place type columns
place_types = [
    'airport', 'american_restaurant', 'amusement_center', 'amusement_park', 'art_gallery',
    'athletic_field', 'atm', 'bakery', 'banquet_hall', 'bar', 'barbecue_restaurant',
    'barber_shop', 'beauty_salon', 'bed_and_breakfast', 'bicycle_store', 'brazilian_restaurant',
    'breakfast_restaurant', 'brunch_restaurant', 'cafe', 'campground', 'camping_cabin',
    'car_rental', 'car_wash', 'cemetery', 'child_care_agency', 'chinese_restaurant', 'church',
    'clothing_store', 'coffee_shop', 'community_center', 'consultant', 'convention_center', 'cottage',
    'cultural_center', 'dog_park', 'event_venue', 'extended_stay_hotel', 'farm', 'farmstay',
    'fast_food_restaurant', 'finance', 'fitness_center', 'florist', 'food', 'french_restaurant',
    'furniture_store', 'general_contractor', 'gift_shop', 'golf_course', 'greek_restaurant',
    'grocery_store', 'guest_house', 'gym', 'hair_care', 'hair_salon', 'hamburger_restaurant',
    'health', 'heliport', 'hiking_area', 'hindu_temple', 'historical_landmark', 'home_goods_store',
    'home_improvement_store', 'hostel', 'hotel', 'ice_cream_shop', 'indian_restaurant',
    'indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'jewelry_store',
    'korean_restaurant', 'lebanese_restaurant', 'library', 'liquor_store', 'lodging', 'market',
    'meal_delivery', 'meal_takeaway', 'mediterranean_restaurant', 'mexican_restaurant',
    'middle_eastern_restaurant', 'mosque', 'motel', 'movie_theater', 'museum', 'national_park',
    'night_club', 'park', 'performing_arts_theater', 'pizza_restaurant',
    'place_of_worship', 'playground', 'preschool', 'private_guest_room', 'ramen_restaurant',
    'real_estate_agency', 'resort_hotel', 'rest_stop', 'restaurant', 'sandwich_shop', 'school',
    'seafood_restaurant', 'shopping_mall', 'spa', 'spanish_restaurant', 'sporting_goods_store',
    'sports_club', 'sports_complex', 'steak_house', 'store', 'supermarket', 'sushi_restaurant',
    'swimming_pool', 'thai_restaurant', 'tourist_attraction', 'travel_agency', 'turkish_restaurant',
    'university', 'vegan_restaurant', 'vegetarian_restaurant', 'vietnamese_restaurant',
    'wedding_venue', 'wholesaler', 'zoo', 'parking'
]

# Sum the place type columns for each user to get their preferences
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()

# Normalize the preferences
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Replace non zero values with 1
user_preferences[place_types] = user_preferences[place_types].applymap(lambda x: 1 if x > 0 else 0)

user_preferences.head(1)


2024-06-20 14:27:28.521278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 14:27:28.521358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 14:27:28.522119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 14:27:28.526456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_314159/3309077594.py:44: FutureWarning

,user_id,airport,american_restaurant,amusement_center,amusement_park,art_gallery,athletic_field,atm,bakery,banquet_hall,...,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo,parking
0,user_1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Hyperparameters
embedding_dim = 50

# User model
user_input = Input(shape=(len(place_types),), name='user_input')
user_embedding = Dense(embedding_dim, activation='relu')(user_input)

# Place model
place_input = Input(shape=(len(place_types),), name='place_input')
place_embedding = Dense(embedding_dim, activation='relu')(place_input)

# Concatenate user and place embeddings
merged = Concatenate()([user_embedding, place_embedding])
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

# Create and compile the model
model = Model(inputs=[user_input, place_input], outputs=output)
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


2024-06-20 14:27:30.285266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 14:27:30.295188: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 14:27:30.295241: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 14:27:30.297737: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 14:27:30.297802: I external/local_xla/xla/stream_executor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 126)]                0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 126)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 50)                   6350      ['user_input[0][0]']          
                                                                                                  
 dense_1 (Dense)             (None, 50)                   6350      ['place_input[0][0]']         
                                                                                              

:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 14:27:30.400061: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3341 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# Ensure user_preferences and data are correctly aligned and split the data accordingly
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Merge data with user preferences to align place features with user preferences
merged_data = user_place_reviews.merge(user_preferences, on='user_id', suffixes=('', '_user'))

# Prepare user features and place features for each review
user_features = merged_data[[f'{ptype}_user' for ptype in place_types]].values
place_features = merged_data[place_types].values

# Generate labels (assuming binary relevance for simplicity)
# You need to ensure your labels are the same length as user and place features
labels = np.random.randint(2, size=(len(user_features),))

# Split into train and test sets ensuring the lengths match
user_train, user_test, place_train, place_test, y_train, y_test = train_test_split(
    user_features, place_features, labels, test_size=0.3, random_state=42
)


In [6]:
# Fit the model
history = model.fit(
    [user_train, place_train],
    y_train,
    epochs=15,
    batch_size=32,
    validation_data=([user_test, place_test], y_test)
)


Epoch 1/15


2024-06-20 14:27:32.113275: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-20 14:27:33.284998: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff0e436b440 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-20 14:27:33.285048: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-06-20 14:27:33.293395: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1718868453.334090  314289 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


676/676 [==============================] - 6s 6ms/step - loss: 0.6933 - accuracy: 0.5032 - val_loss: 0.6933 - val_accuracy: 0.5003
Epoch 2/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6930 - accuracy: 0.5083 - val_loss: 0.6934 - val_accuracy: 0.4918
Epoch 3/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6930 - accuracy: 0.5087 - val_loss: 0.6934 - val_accuracy: 0.5017
Epoch 4/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6929 - accuracy: 0.5071 - val_loss: 0.6935 - val_accuracy: 0.4986
Epoch 5/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6929 - accuracy: 0.5079 - val_loss: 0.6935 - val_accuracy: 0.4930
Epoch 6/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6928 - accuracy: 0.5120 - val_loss: 0.6936 - val_accuracy: 0.4943
Epoch 7/15
676/676 [==============================] - 4s 6ms/step - loss: 0.6928 - accuracy: 0.5124 - val_loss: 0.6935 - val_accuracy: 0.4946
Epoch 8/15
676/67

In [9]:
model.save('user_pref_model.keras')

In [8]:
def recommend_places(selected_types, user_id=None, top_n=20):
    # Get user preferences
    if user_id:
        user_pref = user_preferences[user_preferences['user_id'] == user_id][place_types].values
    else:
        # create a random user preference without sampling
        user_pref = np.random.randint(2, size=(1, len(place_types)))
        
        # user_pref = user_preferences.sample(1)[place_types].values

    filtered_places = user_place_reviews[user_place_reviews[selected_types].any(axis=1)]


    place_features = filtered_places[place_types].drop_duplicates().values
    place_ids = filtered_places['id'].drop_duplicates().values

    predictions = model.predict([np.repeat(user_pref, len(place_features), axis=0), place_features])
    
    top_indices = np.argsort(predictions[:, 0])[-top_n:][::-1]

    recommended_place_ids = place_ids[top_indices]
    
    unique_recommendations = set(recommended_place_ids)
    
  
    recommended_places = filtered_places[filtered_places['id'].isin(unique_recommendations)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]


# select a random user and type
random_user = user_place_reviews['user_id'].sample(1).values[0]
random_type = user_place_reviews['types_x'].sample(1).values[0]



selected_user = 'user_1'  
selected_types = ['hotel', 'lodging']  

# recommendations = recommend_places(user_id=random_user, top_n=10, selected_types=random_type)
recommendations = recommend_places(random_type, top_n=5, user_id=random_user)

print(f"Top Recommendations for {random_user} with types {random_type}:")
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


16/16 [==============================] - 0s 2ms/step
Top Recommendations for user_5834 with types ['restaurant', 'food']:
---------------------------------
Name: Warung Sate Ayam Nyoman
Type: indonesian_restaurant
Rating: 5.0
---------------------------------
Name: Warung D'BUCU
Type: restaurant
Rating: 4.8
---------------------------------
Name: BABI GULING PAK DODOT
Type: restaurant
Rating: 4.6
---------------------------------
Name: Arunika Coffee Eatery
Type: coffee_shop
Rating: 4.5
---------------------------------
Name: Maha Kencana
Type: restaurant
Rating: 3.8
---------------------------------
